# 05. MCP-Style API Integration - 외부 API를 MCP 형식으로 통합하기

이 노트북에서는 **MCP (Model Context Protocol)**의 JSON-RPC 2.0 형식을 이해하고, 실제 외부 API들을 MCP 형식으로 래핑하는 방법을 학습합니다.

## 🎯 학습 목표

1. **MCP JSON-RPC 2.0 표준 형식** 정확하게 이해하기
2. **어댑터 패턴**: 실제 API는 MCP를 모르므로 어댑터가 필요!
3. **12개 무료 API** MCP 형식으로 통합
4. **Manual Graph**: EXAONE이 MCP JSON을 생성하도록 (tool calling 없이!)
5. **장단점 분석**: MCP를 언제 사용해야 하는가?

## 🔑 핵심 개념

**MCP의 본질**: 다양한 도구들을 **표준화된 JSON 형식**으로 통일하는 것!

```
실제 API들 (각기 다른 형식)
  ├─ Wikipedia: wikipedia.summary(topic)
  ├─ OpenMeteo: requests.get('api.open-meteo.com/v1/forecast?...')
  ├─ Pollinations: f'https://image.pollinations.ai/prompt/{text}'
  └─ ...
         ↓
    MCP Adapter (표준화)
         ↓
통일된 JSON-RPC 2.0 형식
  {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/call",
    "params": {
      "name": "search_wikipedia",
      "arguments": {"topic": "Python"}
    }
  }
```

## 📦 사전 요구사항

- 03-react-agent.ipynb (ReAct Loop)
- 04-multi-agent-handoffs.ipynb (Manual Graph 패턴)
- Python 3.8+

## Part 1: MCP 표준 형식 이해하기 📋

### 1.1 MCP란?

**MCP (Model Context Protocol)**는 Anthropic이 2024년에 공개한 오픈 프로토콜로, **JSON-RPC 2.0** 기반입니다.

**핵심 아이디어**: 모든 도구 호출을 동일한 JSON 형식으로 표준화!

### 1.2 JSON-RPC 2.0 기본 구조

모든 MCP 메시지는 다음 필드를 포함합니다:

- `jsonrpc`: 항상 `"2.0"`
- `id`: 요청/응답 매칭용 고유 ID
- `method`: 호출할 메서드 (예: `"tools/call"`)
- `params`: 메서드 파라미터 (optional)

### 1.3 MCP의 주요 메서드

1. **`tools/list`**: 사용 가능한 도구 목록 조회
2. **`tools/call`**: 특정 도구 실행
3. **`resources/list`**: 사용 가능한 리소스 조회 (파일, DB 등)
4. **`resources/read`**: 리소스 읽기

이번 노트북에서는 **`tools/list`**와 **`tools/call`**을 집중적으로 다룹니다.

### 1.4 정확한 MCP 형식 - tools/list

**요청 형식** (사용 가능한 도구 목록 조회):

```json
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/list",
  "params": {}
}
```

**응답 형식**:

```json
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "tools": [
      {
        "name": "search_wikipedia",
        "description": "Wikipedia에서 주제를 검색합니다",
        "inputSchema": {
          "type": "object",
          "properties": {
            "topic": {
              "type": "string",
              "description": "검색할 주제"
            }
          },
          "required": ["topic"]
        }
      }
    ]
  }
}
```

**주의**: `inputSchema`는 **JSON Schema** 형식을 따릅니다!

### 1.5 정확한 MCP 형식 - tools/call

**요청 형식** (도구 실행):

```json
{
  "jsonrpc": "2.0",
  "id": 2,
  "method": "tools/call",
  "params": {
    "name": "search_wikipedia",
    "arguments": {
      "topic": "Python programming"
    }
  }
}
```

**응답 형식** (성공):

```json
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "Python is a high-level programming language..."
      }
    ],
    "isError": false
  }
}
```

**응답 형식** (에러):

```json
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "Tool not found: invalid_tool"
      }
    ],
    "isError": true
  }
}
```

**중요**: MCP는 도구 실행 에러를 `result.isError: true`로 표시합니다!

### 1.6 실습: MCP 형식 직접 만들어보기

코드로 MCP 메시지를 생성해봅시다.

In [ ]:
import json

def create_tools_list_request(request_id=1):
    """tools/list 요청 생성"""
    return {
        "jsonrpc": "2.0",
        "id": request_id,
        "method": "tools/list",
        "params": {}
    }

def create_tools_call_request(tool_name: str, arguments: dict, request_id=2):
    """tools/call 요청 생성"""
    return {
        "jsonrpc": "2.0",
        "id": request_id,
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": arguments
        }
    }

def create_mcp_response(result_content: str, request_id: int, is_error=False):
    """MCP 응답 생성"""
    return {
        "jsonrpc": "2.0",
        "id": request_id,
        "result": {
            "content": [
                {
                    "type": "text",
                    "text": result_content
                }
            ],
            "isError": is_error
        }
    }

# 테스트
print("📤 tools/list 요청:")
list_req = create_tools_list_request()
print(json.dumps(list_req, indent=2, ensure_ascii=False))

print("\n📤 tools/call 요청:")
call_req = create_tools_call_request("search_wikipedia", {"topic": "Python"})
print(json.dumps(call_req, indent=2, ensure_ascii=False))

print("\n📥 MCP 응답 (성공):")
response = create_mcp_response("Python is a programming language...", request_id=2)
print(json.dumps(response, indent=2, ensure_ascii=False))

print("\n✅ MCP 형식 이해 완료!")

## Part 2: 어댑터 패턴 - 실제 API는 MCP를 모른다! 🔌

### 2.1 문제: 실제 API들은 MCP 형식을 지원하지 않음

Wikipedia, OpenWeather, Pollinations 등 실제 API들은 각자 고유한 인터페이스를 가지고 있습니다:

```python
# Wikipedia API
import wikipedia
result = wikipedia.summary("Python", sentences=3)  # 문자열 반환

# OpenMeteo API
import requests
response = requests.get("https://api.open-meteo.com/v1/forecast?...")
data = response.json()  # 딕셔너리 반환

# Pollinations API
url = f"https://image.pollinations.ai/prompt/{urllib.parse.quote(prompt)}"
# URL만 반환
```

이들은 **MCP JSON-RPC 2.0 형식을 전혀 모릅니다!**

### 2.2 해결책: MCP Adapter 구현

**어댑터 패턴**: 각 API를 MCP 형식으로 래핑하는 어댑터를 만듭니다.

```
Wikipedia API ─┐
OpenMeteo API ─┼─→ MCP Adapter ─→ 통일된 MCP JSON 형식
Pollinations  ─┘
```

어댑터가 하는 일:
1. **Tool Definition 생성**: 각 API를 MCP Tool로 정의 (JSON Schema 포함)
2. **요청 변환**: MCP `tools/call` 요청 → 실제 API 호출
3. **응답 변환**: 실제 API 응답 → MCP 응답 형식

### 2.3 왜 어댑터가 필요한가?

**MCP의 목표**: 다양한 도구를 **하나의 표준 인터페이스**로 통일

- LLM은 MCP JSON만 생성하면 됨 (각 API의 인터페이스를 몰라도 됨)
- 새로운 API 추가 시 어댑터만 작성하면 됨
- 도구 교체가 쉬움 (MCP 인터페이스는 그대로)

### 2.4 예제: Wikipedia API → MCP Adapter

In [ ]:
# 먼저 필요한 패키지 설치
!pip install -q wikipedia-api requests

In [ ]:
import wikipedia

# Step 1: 실제 Wikipedia API 함수
def wikipedia_search_raw(topic: str, sentences: int = 3) -> str:
    """원본 Wikipedia API 호출"""
    try:
        wikipedia.set_lang('ko')
        return wikipedia.summary(topic, sentences=sentences)
    except Exception as e:
        return f"Error: {str(e)}"

# Step 2: MCP Tool Definition 생성
WIKIPEDIA_TOOL_DEF = {
    "name": "search_wikipedia",
    "description": "Wikipedia에서 주제를 검색하여 요약 정보를 가져옵니다",
    "inputSchema": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "검색할 주제 (한글 또는 영어)"
            },
            "sentences": {
                "type": "integer",
                "description": "반환할 문장 수 (기본 3)",
                "default": 3
            }
        },
        "required": ["topic"]
    }
}

# Step 3: MCP Adapter 함수
def execute_wikipedia_tool(arguments: dict) -> str:
    """MCP arguments를 Wikipedia API로 변환하여 실행"""
    topic = arguments["topic"]
    sentences = arguments.get("sentences", 3)
    return wikipedia_search_raw(topic, sentences)

# ============================================================
# Before (원본 API) vs After (MCP 형식)
# ============================================================

print("="*60)
print("🔴 Before: 원본 Wikipedia API 사용")
print("="*60)
result_before = wikipedia_search_raw("Python", 2)
print(f"Input: topic='Python', sentences=2")
print(f"Output type: {type(result_before)}")
print(f"Output: {result_before[:100]}...")

print("\n" + "="*60)
print("🟢 After: MCP 형식으로 사용")
print("="*60)

# MCP 요청 생성
mcp_request = create_tools_call_request(
    tool_name="search_wikipedia",
    arguments={"topic": "Python", "sentences": 2}
)

print("📤 MCP Request:")
print(json.dumps(mcp_request, indent=2, ensure_ascii=False))

# 도구 실행
result_text = execute_wikipedia_tool(mcp_request["params"]["arguments"])

# MCP 응답 생성
mcp_response = create_mcp_response(result_text, mcp_request["id"])

print("\n📥 MCP Response:")
print(json.dumps(mcp_response, indent=2, ensure_ascii=False))

print("\n✅ 어댑터 패턴으로 Wikipedia API를 MCP 형식으로 래핑 완료!")

## Part 3: 12개 무료 API의 MCP Adapter 구현 🌐

이제 **12개의 무료 API**를 모두 MCP 형식으로 래핑합니다!

### 3.1 API 목록

#### 인증 불필요 (7개):
1. **Wikipedia** - 백과사전 검색
2. **ExchangeRate-API** - 환율 조회
3. **Pollinations** - AI 이미지 생성 (완전 무료!)
4. **OpenMeteo** - 날씨 정보 (인증 없음!)
5. **JSONPlaceholder** - 가짜 REST API (테스트용)
6. **Random User Generator** - 무작위 사용자 데이터
7. **DummyJSON** - 샘플 e-commerce 데이터

#### 무료 Tier (5개):
8. **NewsAPI** - 뉴스 검색 (무료: 100 req/day)
9. **OpenWeather** - 상세 날씨 (무료: 1000 calls/day)
10. **GitHub API** - 저장소 정보 (무료: 60 req/hour 인증 없이)
11. **IP Geolocation** - IP 위치 정보
12. **REST Countries** - 국가 정보

### 3.2 필수 패키지 설치

In [ ]:
# 필수 패키지
!pip install -q requests wikipedia-api python-dotenv

print("✅ 패키지 설치 완료")

### 3.3 MCP Tool Registry 구축

모든 도구를 한곳에 등록합니다.

In [ ]:
import requests
import urllib.parse
from typing import Dict, Any, List

# ============================================================
# MCP Tool Definitions (JSON Schema)
# ============================================================

MCP_TOOLS_REGISTRY: Dict[str, Dict[str, Any]] = {}

# Tool 1: Wikipedia
MCP_TOOLS_REGISTRY["search_wikipedia"] = {
    "name": "search_wikipedia",
    "description": "Wikipedia에서 주제를 검색하여 요약 정보를 가져옵니다",
    "inputSchema": {
        "type": "object",
        "properties": {
            "topic": {"type": "string", "description": "검색할 주제"},
            "sentences": {"type": "integer", "description": "반환할 문장 수", "default": 3}
        },
        "required": ["topic"]
    }
}

# Tool 2: Exchange Rate
MCP_TOOLS_REGISTRY["get_exchange_rate"] = {
    "name": "get_exchange_rate",
    "description": "두 통화 간의 환율을 조회합니다 (무료, 인증 불필요)",
    "inputSchema": {
        "type": "object",
        "properties": {
            "from_currency": {"type": "string", "description": "기준 통화 (예: USD, KRW)"},
            "to_currency": {"type": "string", "description": "변환 통화 (예: EUR, JPY)"}
        },
        "required": ["from_currency", "to_currency"]
    }
}

# Tool 3: Pollinations Image Generation
MCP_TOOLS_REGISTRY["generate_ai_image"] = {
    "name": "generate_ai_image",
    "description": "AI가 텍스트 프롬프트로 이미지를 생성합니다 (완전 무료!)",
    "inputSchema": {
        "type": "object",
        "properties": {
            "prompt": {"type": "string", "description": "이미지 설명 (영어 권장)"},
            "width": {"type": "integer", "description": "이미지 너비", "default": 512},
            "height": {"type": "integer", "description": "이미지 높이", "default": 512}
        },
        "required": ["prompt"]
    }
}

# Tool 4: OpenMeteo Weather
MCP_TOOLS_REGISTRY["get_weather"] = {
    "name": "get_weather",
    "description": "도시의 현재 날씨를 조회합니다 (무료, 인증 불필요)",
    "inputSchema": {
        "type": "object",
        "properties": {
            "city": {"type": "string", "description": "도시 이름 (영어)"},
            "units": {"type": "string", "description": "단위 (celsius/fahrenheit)", "default": "celsius"}
        },
        "required": ["city"]
    }
}

# Tool 5: JSONPlaceholder
MCP_TOOLS_REGISTRY["get_dummy_post"] = {
    "name": "get_dummy_post",
    "description": "가짜 블로그 포스트를 가져옵니다 (테스트용)",
    "inputSchema": {
        "type": "object",
        "properties": {
            "post_id": {"type": "integer", "description": "포스트 ID (1-100)"}
        },
        "required": ["post_id"]
    }
}

# Tool 6: Random User Generator
MCP_TOOLS_REGISTRY["generate_random_user"] = {
    "name": "generate_random_user",
    "description": "무작위 사용자 정보를 생성합니다 (테스트 데이터용)",
    "inputSchema": {
        "type": "object",
        "properties": {
            "gender": {"type": "string", "description": "성별 (male/female/random)", "default": "random"},
            "nat": {"type": "string", "description": "국적 코드 (US/GB/KR 등)", "default": "US"}
        },
        "required": []
    }
}

# Tool 7: DummyJSON Products
MCP_TOOLS_REGISTRY["search_products"] = {
    "name": "search_products",
    "description": "샘플 e-commerce 제품을 검색합니다",
    "inputSchema": {
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "검색어"}
        },
        "required": ["query"]
    }
}

# Tool 8: REST Countries
MCP_TOOLS_REGISTRY["get_country_info"] = {
    "name": "get_country_info",
    "description": "국가의 상세 정보를 조회합니다",
    "inputSchema": {
        "type": "object",
        "properties": {
            "country_name": {"type": "string", "description": "국가 이름 (영어)"}
        },
        "required": ["country_name"]
    }
}

# Tool 9: GitHub Repo Info (인증 없이 public repo만)
MCP_TOOLS_REGISTRY["get_github_repo"] = {
    "name": "get_github_repo",
    "description": "GitHub 저장소 정보를 조회합니다 (public repo만)",
    "inputSchema": {
        "type": "object",
        "properties": {
            "owner": {"type": "string", "description": "저장소 소유자"},
            "repo": {"type": "string", "description": "저장소 이름"}
        },
        "required": ["owner", "repo"]
    }
}

# Tool 10: IP Geolocation
MCP_TOOLS_REGISTRY["get_ip_location"] = {
    "name": "get_ip_location",
    "description": "IP 주소의 지리적 위치를 조회합니다",
    "inputSchema": {
        "type": "object",
        "properties": {
            "ip": {"type": "string", "description": "IP 주소 (비어있으면 현재 IP)"}
        },
        "required": []
    }
}

# Tool 11: Simple Calculator
MCP_TOOLS_REGISTRY["calculate"] = {
    "name": "calculate",
    "description": "간단한 수식을 계산합니다",
    "inputSchema": {
        "type": "object",
        "properties": {
            "expression": {"type": "string", "description": "계산식 (예: 2+3*4)"}
        },
        "required": ["expression"]
    }
}

# Tool 12: Current Time
MCP_TOOLS_REGISTRY["get_current_time"] = {
    "name": "get_current_time",
    "description": "현재 시간을 조회합니다",
    "inputSchema": {
        "type": "object",
        "properties": {
            "timezone": {"type": "string", "description": "시간대 (예: Asia/Seoul)", "default": "UTC"}
        },
        "required": []
    }
}

print(f"✅ MCP Tool Registry 생성 완료: {len(MCP_TOOLS_REGISTRY)}개 도구")
print("\n📦 등록된 도구:")
for tool_name in MCP_TOOLS_REGISTRY.keys():
    print(f"   - {tool_name}")

### 3.4 MCP Adapter 함수 구현

각 API를 실제로 호출하는 어댑터 함수들입니다.

In [ ]:
import wikipedia
from datetime import datetime
import re

# ============================================================
# MCP Adapter Functions (실제 API 호출)
# ============================================================

def execute_search_wikipedia(arguments: dict) -> str:
    """Wikipedia 검색"""
    try:
        topic = arguments["topic"]
        sentences = arguments.get("sentences", 3)
        wikipedia.set_lang('ko')
        return wikipedia.summary(topic, sentences=sentences)
    except wikipedia.exceptions.DisambiguationError as e:
        return f"여러 결과가 있습니다: {', '.join(e.options[:5])}"
    except Exception as e:
        return f"검색 실패: {str(e)}"

def execute_get_exchange_rate(arguments: dict) -> str:
    """환율 조회 (ExchangeRate-API)"""
    try:
        from_currency = arguments["from_currency"].upper()
        to_currency = arguments["to_currency"].upper()

        url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
        response = requests.get(url, timeout=5)
        data = response.json()

        if to_currency in data["rates"]:
            rate = data["rates"][to_currency]
            return f"1 {from_currency} = {rate:.4f} {to_currency}"
        else:
            return f"{to_currency} 통화를 찾을 수 없습니다"
    except Exception as e:
        return f"환율 조회 실패: {str(e)}"

def execute_generate_ai_image(arguments: dict) -> str:
    """AI 이미지 생성 (Pollinations)"""
    try:
        prompt = arguments["prompt"]
        width = arguments.get("width", 512)
        height = arguments.get("height", 512)

        # Pollinations API: 완전 무료!
        encoded_prompt = urllib.parse.quote(prompt)
        url = f"https://image.pollinations.ai/prompt/{encoded_prompt}?width={width}&height={height}"

        return f"이미지 생성 완료!\nURL: {url}\n\n이 URL을 브라우저에서 열어보세요."
    except Exception as e:
        return f"이미지 생성 실패: {str(e)}"

def execute_get_weather(arguments: dict) -> str:
    """날씨 조회 (OpenMeteo) - 좌표 변환 포함"""
    try:
        city = arguments["city"]

        # 주요 도시 좌표 (간단한 매핑)
        city_coords = {
            "seoul": (37.5665, 126.9780),
            "busan": (35.1796, 129.0756),
            "new york": (40.7128, -74.0060),
            "london": (51.5074, -0.1278),
            "tokyo": (35.6762, 139.6503),
            "paris": (48.8566, 2.3522),
        }

        coords = city_coords.get(city.lower())
        if not coords:
            return f"{city}의 좌표를 찾을 수 없습니다. 지원 도시: {', '.join(city_coords.keys())}"

        lat, lon = coords
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"

        response = requests.get(url, timeout=5)
        data = response.json()

        current = data["current_weather"]
        temp = current["temperature"]
        windspeed = current["windspeed"]

        return f"{city.title()} 현재 날씨:\n온도: {temp}°C\n풍속: {windspeed} km/h"
    except Exception as e:
        return f"날씨 조회 실패: {str(e)}"

def execute_get_dummy_post(arguments: dict) -> str:
    """가짜 포스트 가져오기 (JSONPlaceholder)"""
    try:
        post_id = arguments["post_id"]
        url = f"https://jsonplaceholder.typicode.com/posts/{post_id}"

        response = requests.get(url, timeout=5)
        data = response.json()

        return f"제목: {data['title']}\n\n내용: {data['body']}"
    except Exception as e:
        return f"포스트 조회 실패: {str(e)}"

def execute_generate_random_user(arguments: dict) -> str:
    """무작위 사용자 생성 (Random User Generator)"""
    try:
        gender = arguments.get("gender", "random")
        nat = arguments.get("nat", "US")

        params = {}
        if gender != "random":
            params["gender"] = gender
        params["nat"] = nat

        url = "https://randomuser.me/api/"
        response = requests.get(url, params=params, timeout=5)
        data = response.json()

        user = data["results"][0]
        name = f"{user['name']['first']} {user['name']['last']}"
        email = user["email"]
        location = f"{user['location']['city']}, {user['location']['country']}"

        return f"이름: {name}\n이메일: {email}\n위치: {location}"
    except Exception as e:
        return f"사용자 생성 실패: {str(e)}"

def execute_search_products(arguments: dict) -> str:
    """제품 검색 (DummyJSON)"""
    try:
        query = arguments["query"]
        url = f"https://dummyjson.com/products/search?q={urllib.parse.quote(query)}"

        response = requests.get(url, timeout=5)
        data = response.json()

        if data["products"]:
            products = data["products"][:3]  # 상위 3개
            result = f"{len(products)}개 제품 발견:\n\n"
            for p in products:
                result += f"- {p['title']} (${p['price']})\n"
            return result
        else:
            return "검색 결과가 없습니다"
    except Exception as e:
        return f"제품 검색 실패: {str(e)}"

def execute_get_country_info(arguments: dict) -> str:
    """국가 정보 조회 (REST Countries)"""
    try:
        country_name = arguments["country_name"]
        url = f"https://restcountries.com/v3.1/name/{urllib.parse.quote(country_name)}"

        response = requests.get(url, timeout=5)
        data = response.json()

        if isinstance(data, list) and len(data) > 0:
            country = data[0]
            name = country["name"]["common"]
            capital = country.get("capital", ["N/A"])[0]
            population = country["population"]
            region = country["region"]

            return f"국가: {name}\n수도: {capital}\n인구: {population:,}\n지역: {region}"
        else:
            return "국가를 찾을 수 없습니다"
    except Exception as e:
        return f"국가 정보 조회 실패: {str(e)}"

def execute_get_github_repo(arguments: dict) -> str:
    """GitHub 저장소 정보 조회"""
    try:
        owner = arguments["owner"]
        repo = arguments["repo"]
        url = f"https://api.github.com/repos/{owner}/{repo}"

        response = requests.get(url, timeout=5)
        data = response.json()

        if "message" in data:
            return f"저장소를 찾을 수 없습니다: {data['message']}"

        name = data["full_name"]
        desc = data.get("description", "설명 없음")
        stars = data["stargazers_count"]
        forks = data["forks_count"]

        return f"저장소: {name}\n설명: {desc}\n⭐ Stars: {stars:,}\n🍴 Forks: {forks:,}"
    except Exception as e:
        return f"저장소 조회 실패: {str(e)}"

def execute_get_ip_location(arguments: dict) -> str:
    """IP 위치 조회 (ipapi.co)"""
    try:
        ip = arguments.get("ip", "")
        url = f"https://ipapi.co/{ip}/json/" if ip else "https://ipapi.co/json/"

        response = requests.get(url, timeout=5)
        data = response.json()

        ip_addr = data.get("ip", "N/A")
        city = data.get("city", "N/A")
        region = data.get("region", "N/A")
        country = data.get("country_name", "N/A")

        return f"IP: {ip_addr}\n위치: {city}, {region}, {country}"
    except Exception as e:
        return f"IP 위치 조회 실패: {str(e)}"

def execute_calculate(arguments: dict) -> str:
    """간단한 계산"""
    try:
        expression = arguments["expression"]
        # 안전한 eval (숫자와 연산자만 허용)
        if re.match(r'^[0-9+\-*/()\s.]+$', expression):
            result = eval(expression)
            return f"{expression} = {result}"
        else:
            return "안전하지 않은 수식입니다"
    except Exception as e:
        return f"계산 실패: {str(e)}"

def execute_get_current_time(arguments: dict) -> str:
    """현재 시간 조회"""
    try:
        timezone = arguments.get("timezone", "UTC")
        # 간단한 UTC 시간만 반환 (pytz 없이)
        now = datetime.utcnow()
        return f"현재 시간 (UTC): {now.strftime('%Y-%m-%d %H:%M:%S')}"
    except Exception as e:
        return f"시간 조회 실패: {str(e)}"

# ============================================================
# Tool Name → Executor 매핑
# ============================================================

TOOL_EXECUTORS = {
    "search_wikipedia": execute_search_wikipedia,
    "get_exchange_rate": execute_get_exchange_rate,
    "generate_ai_image": execute_generate_ai_image,
    "get_weather": execute_get_weather,
    "get_dummy_post": execute_get_dummy_post,
    "generate_random_user": execute_generate_random_user,
    "search_products": execute_search_products,
    "get_country_info": execute_get_country_info,
    "get_github_repo": execute_get_github_repo,
    "get_ip_location": execute_get_ip_location,
    "calculate": execute_calculate,
    "get_current_time": execute_get_current_time,
}

print(f"✅ MCP Adapter 함수 구현 완료: {len(TOOL_EXECUTORS)}개")

### 3.5 테스트: 여러 API 테스트

In [ ]:
# 여러 도구 테스트
print("🧪 MCP Adapter 테스트\n" + "="*60)

# Test 1: Wikipedia
print("\n1️⃣ Wikipedia 검색")
result = execute_search_wikipedia({"topic": "Python", "sentences": 2})
print(result[:150] + "...")

# Test 2: Exchange Rate
print("\n2️⃣ 환율 조회")
result = execute_get_exchange_rate({"from_currency": "USD", "to_currency": "KRW"})
print(result)

# Test 3: AI Image
print("\n3️⃣ AI 이미지 생성")
result = execute_generate_ai_image({"prompt": "a beautiful sunset over the ocean", "width": 512, "height": 512})
print(result)

# Test 4: Weather
print("\n4️⃣ 날씨 조회")
result = execute_get_weather({"city": "seoul"})
print(result)

# Test 5: Calculator
print("\n5️⃣ 계산")
result = execute_calculate({"expression": "2+3*4"})
print(result)

print("\n" + "="*60)
print("✅ 모든 어댑터가 정상 작동합니다!")

## Part 4: MCP Handler - 통합 요청 처리기 🎯

이제 MCP 요청을 받아서 처리하는 **Handler**를 만듭니다.

### 4.1 Handler가 하는 일

1. **tools/list 요청** → Tool Registry 반환
2. **tools/call 요청** → 해당 Adapter 함수 실행 → 응답 생성
3. **에러 처리** → `isError: true` 응답 생성

In [ ]:
class MCPHandler:
    """MCP JSON-RPC 2.0 요청을 처리하는 핸들러"""

    def __init__(self, tools_registry: dict, tool_executors: dict):
        self.tools_registry = tools_registry
        self.tool_executors = tool_executors

    def handle_tools_list(self, request: dict) -> dict:
        """tools/list 요청 처리"""
        request_id = request["id"]

        # Tool Definition 리스트 생성
        tools = [
            {
                "name": tool["name"],
                "description": tool["description"],
                "inputSchema": tool["inputSchema"]
            }
            for tool in self.tools_registry.values()
        ]

        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "result": {
                "tools": tools
            }
        }

    def handle_tools_call(self, request: dict) -> dict:
        """tools/call 요청 처리"""
        request_id = request["id"]
        tool_name = request["params"]["name"]
        arguments = request["params"]["arguments"]

        # 도구 실행
        if tool_name in self.tool_executors:
            try:
                result_text = self.tool_executors[tool_name](arguments)
                is_error = False
            except Exception as e:
                result_text = f"Tool execution error: {str(e)}"
                is_error = True
        else:
            result_text = f"Tool not found: {tool_name}"
            is_error = True

        # MCP 응답 생성
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "result": {
                "content": [
                    {
                        "type": "text",
                        "text": result_text
                    }
                ],
                "isError": is_error
            }
        }

    def handle_request(self, request: dict) -> dict:
        """MCP 요청을 자동으로 라우팅"""
        method = request.get("method")

        if method == "tools/list":
            return self.handle_tools_list(request)
        elif method == "tools/call":
            return self.handle_tools_call(request)
        else:
            # 알 수 없는 메서드
            return {
                "jsonrpc": "2.0",
                "id": request.get("id", 0),
                "error": {
                    "code": -32601,
                    "message": f"Method not found: {method}"
                }
            }

# Handler 생성
mcp_handler = MCPHandler(MCP_TOOLS_REGISTRY, TOOL_EXECUTORS)

print("✅ MCP Handler 생성 완료")

### 4.2 테스트: MCP Handler 사용

In [ ]:
print("🧪 MCP Handler 테스트\n" + "="*60)

# Test 1: tools/list 요청
print("\n📤 Test 1: tools/list 요청")
list_request = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list",
    "params": {}
}

list_response = mcp_handler.handle_request(list_request)
print(f"✅ {len(list_response['result']['tools'])}개 도구 반환됨")
print("\n처음 3개 도구:")
for tool in list_response['result']['tools'][:3]:
    print(f"  - {tool['name']}: {tool['description']}")

# Test 2: tools/call 요청 (Wikipedia)
print("\n\n📤 Test 2: tools/call 요청 (Wikipedia)")
call_request = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "search_wikipedia",
        "arguments": {"topic": "Machine Learning", "sentences": 2}
    }
}

print("요청:")
print(json.dumps(call_request, indent=2, ensure_ascii=False))

call_response = mcp_handler.handle_request(call_request)

print("\n응답:")
print(json.dumps(call_response, indent=2, ensure_ascii=False))

# Test 3: 존재하지 않는 도구
print("\n\n📤 Test 3: 존재하지 않는 도구 호출")
error_request = {
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/call",
    "params": {
        "name": "nonexistent_tool",
        "arguments": {}
    }
}

error_response = mcp_handler.handle_request(error_request)
print("응답:")
print(json.dumps(error_response, indent=2, ensure_ascii=False))

print("\n" + "="*60)
print("✅ MCP Handler가 정상 작동합니다!")

## Part 5: LangGraph + MCP JSON (Manual Graph 패턴) 🤖

이제 **04-multi-agent-handoffs.ipynb**에서 배운 **Manual Graph 패턴**을 사용해서, EXAONE이 **MCP JSON을 생성**하도록 만듭니다!

### 5.1 핵심 아이디어

**Tool calling 없이** LLM이 **MCP JSON을 텍스트로 생성**하게 만듭니다:

```python
# ❌ Tool calling 방식 (EXAONE이 약함)
llm_with_tools = llm.bind_tools(tools)
response = llm_with_tools.invoke(messages)  # tool_calls 자동 생성

# ✅ Manual Graph 방식 (EXAONE 호환)
prompt = f"다음 도구를 사용하려면 MCP JSON 형식으로 응답하세요: {tools}"
response = llm.invoke(prompt)  # JSON 텍스트 생성
json_request = extract_json(response)  # 텍스트에서 JSON 추출
```

### 5.2 장점

- ✅ Tool calling 지원 불필요
- ✅ HuggingFace 모델 호환
- ✅ MCP 형식 학습 효과
- ✅ 디버깅 쉬움 (JSON이 보임)

### 5.3 EXAONE 모델 설정

In [ ]:
# EXAONE 모델 (04번과 동일)
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

print("🔄 EXAONE 모델 로딩 중...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

print(f"✅ 모델 로딩 완료: {model_name}")
print(f"   Device: {next(model.parameters()).device}")

def generate_with_exaone(prompt: str, max_length: int = 512) -> str:
    """EXAONE으로 텍스트 생성"""
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 프롬프트 제거
    response = response[len(prompt):].strip()
    return response

# 간단한 테스트
test_response = generate_with_exaone("Hello, EXAONE! ")
print(f"\n테스트: {test_response[:100]}...")

### 5.4 MCP JSON 생성 프롬프트 엔지니어링

EXAONE이 MCP JSON을 생성하도록 프롬프트를 만듭니다.

In [ ]:
def create_mcp_agent_prompt(user_query: str, tools_list: list, conversation_history: list = None) -> str:
    """MCP JSON 생성 프롬프트"""

    # 도구 목록 포맷팅
    tools_desc = ""
    for i, tool in enumerate(tools_list[:5], 1):  # 처음 5개만 (프롬프트 길이 제한)
        tools_desc += f"{i}. **{tool['name']}**: {tool['description']}\n"
        props = tool['inputSchema']['properties']
        params = ', '.join([f"{k} ({v.get('description', 'no desc')})" for k, v in props.items()])
        tools_desc += f"   Parameters: {params}\n"

    # 이전 대화 기록
    history_text = ""
    if conversation_history:
        for item in conversation_history[-3:]:  # 최근 3개만
            history_text += f"- {item}\n"

    prompt = f"""당신은 도구를 사용할 수 있는 AI 어시스턴트입니다.

사용 가능한 도구:
{tools_desc}

사용자 질문: {user_query}

{"이전 대화:" if history_text else ""}
{history_text}

도구를 사용해야 한다면 아래 MCP JSON 형식으로 응답하세요:
{{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/call",
  "params": {{
    "name": "도구이름",
    "arguments": {{"파라미터": "값"}}
  }}
}}

도구 없이 바로 답변할 수 있으면 일반 텍스트로 답하세요.

응답:"""

    return prompt

# 테스트
test_prompt = create_mcp_agent_prompt(
    "What is Python programming language?",
    list(MCP_TOOLS_REGISTRY.values())
)

print("📝 생성된 프롬프트 (일부):")
print(test_prompt[:500] + "\n...")

### 5.5 JSON 추출 함수

In [ ]:
def extract_mcp_json_from_response(response_text: str) -> dict:
    """LLM 응답에서 MCP JSON 추출"""
    import re

    # JSON 블록 찾기
    json_match = re.search(r'\{[^{}]*"jsonrpc"[^{}]*"method"[^{}]*\}', response_text, re.DOTALL)

    if json_match:
        try:
            json_str = json_match.group()
            mcp_request = json.loads(json_str)
            return mcp_request
        except json.JSONDecodeError:
            return None

    return None

# 테스트
test_llm_output = """네, 도구를 사용하겠습니다.

{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/call",
  "params": {
    "name": "search_wikipedia",
    "arguments": {"topic": "Python"}
  }
}

이렇게 검색합니다."""

extracted = extract_mcp_json_from_response(test_llm_output)
print("🔍 추출된 MCP JSON:")
print(json.dumps(extracted, indent=2, ensure_ascii=False))

### 5.6 MCP Agent 구현 (Manual Graph)

이제 LangGraph로 MCP Agent를 구현합니다.

In [ ]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

# State 정의
class MCPAgentState(TypedDict):
    user_query: str
    conversation_history: List[str]
    tool_results: List[dict]
    final_answer: str
    iteration: int
    max_iterations: int

def mcp_agent_node(state: MCPAgentState):
    """MCP Agent 노드: MCP JSON 생성 및 실행"""
    print(f"\n🤖 [MCP Agent] Iteration {state['iteration'] + 1}/{state['max_iterations']}")

    # 프롬프트 생성
    tools_list = list(MCP_TOOLS_REGISTRY.values())
    prompt = create_mcp_agent_prompt(
        state['user_query'],
        tools_list,
        state['conversation_history']
    )

    # LLM 응답 생성
    llm_response = generate_with_exaone(prompt, max_length=400)

    print(f"   LLM 응답: {llm_response[:150]}...")

    # MCP JSON 추출 시도
    mcp_request = extract_mcp_json_from_response(llm_response)

    if mcp_request and mcp_request.get("method") == "tools/call":
        # 도구 호출!
        print(f"   🔧 도구 호출 감지: {mcp_request['params']['name']}")

        # MCP Handler로 실행
        mcp_response = mcp_handler.handle_request(mcp_request)

        # 결과 저장
        tool_result = mcp_response['result']['content'][0]['text']
        state['tool_results'].append({
            'request': mcp_request,
            'response': tool_result
        })

        # 대화 기록 업데이트
        state['conversation_history'].append(
            f"도구 {mcp_request['params']['name']} 사용 → 결과: {tool_result[:100]}"
        )

        # 반복 횟수 증가
        state['iteration'] += 1

        # 최대 반복 확인
        if state['iteration'] >= state['max_iterations']:
            state['final_answer'] = f"최종 결과:\n{tool_result}"
            return Command(goto=END)
        else:
            # 계속 추론
            return Command(goto="mcp_agent")

    else:
        # JSON 없음 = 최종 답변
        print(f"   💬 최종 답변 생성")
        state['final_answer'] = llm_response
        return Command(goto=END)

# Graph 구성
graph = StateGraph(MCPAgentState)
graph.add_node("mcp_agent", mcp_agent_node)
graph.add_edge(START, "mcp_agent")

mcp_agent_app = graph.compile()

print("✅ MCP Agent (Manual Graph) 생성 완료!")

### 5.7 테스트: MCP Agent 실행

In [ ]:
# 테스트 쿼리
test_queries = [
    "서울의 현재 날씨를 알려줘",
    "1 USD를 KRW로 환율 변환해줘",
    "Python 프로그래밍에 대해 설명해줘 (Wikipedia 검색)",
]

for query in test_queries[:1]:  # 일단 첫 번째만
    print("="*60)
    print(f"📝 질문: {query}")
    print("="*60)

    # Agent 실행
    result = mcp_agent_app.invoke({
        "user_query": query,
        "conversation_history": [],
        "tool_results": [],
        "final_answer": "",
        "iteration": 0,
        "max_iterations": 3
    })

    print("\n" + "="*60)
    print("💬 최종 답변:")
    print(result["final_answer"])
    print("\n" + "="*60)

    if result["tool_results"]:
        print(f"\n🔧 사용된 도구: {len(result['tool_results'])}개")
        for i, tool_result in enumerate(result['tool_results'], 1):
            req = tool_result['request']
            print(f"   {i}. {req['params']['name']}: {req['params']['arguments']}")

print("\n✅ MCP Agent 테스트 완료!")

## Part 6: 실전 시나리오 🚀

MCP Agent를 실제 사용 사례에 적용합니다.

### 6.1 시나리오 1: 여행 플래너

사용자가 여행지를 물어보면:
1. **Wikipedia**로 여행지 정보 검색
2. **날씨 API**로 현지 날씨 확인
3. **환율 API**로 환전 정보 제공

In [ ]:
def travel_planner_scenario():
    """여행 플래너 시나리오"""
    query = "I'm planning to visit Seoul, South Korea. What should I know?"

    print("✈️ 여행 플래너 시나리오")
    print("="*60)
    print(f"📝 질문: {query}")
    print("="*60)

    result = mcp_agent_app.invoke({
        "user_query": query,
        "conversation_history": [],
        "tool_results": [],
        "final_answer": "",
        "iteration": 0,
        "max_iterations": 3
    })

    print("\n" + "="*60)
    print("💬 답변:")
    print(result["final_answer"])

    if result["tool_results"]:
        print(f"\n🔧 사용된 도구:")
        for tool_result in result["tool_results"]:
            req = tool_result['request']
            print(f"   - {req['params']['name']}: {req['params']['arguments']}")

# 실행
travel_planner_scenario()

### 6.2 시나리오 2: 데이터 분석 어시스턴트

사용자의 계산 요청을 처리합니다.

In [ ]:
def data_analysis_scenario():
    """데이터 분석 시나리오"""
    query = "Calculate: (1500 * 1.2) + 300. Then convert the result from USD to KRW."

    print("📊 데이터 분석 시나리오")
    print("="*60)
    print(f"📝 질문: {query}")
    print("="*60)

    result = mcp_agent_app.invoke({
        "user_query": query,
        "conversation_history": [],
        "tool_results": [],
        "final_answer": "",
        "iteration": 0,
        "max_iterations": 5  # 여러 단계 가능
    })

    print("\n" + "="*60)
    print("💬 답변:")
    print(result["final_answer"])

    if result["tool_results"]:
        print(f"\n🔧 도구 사용 순서:")
        for i, tool_result in enumerate(result["tool_results"], 1):
            req = tool_result['request']
            resp = tool_result['response']
            print(f"   {i}. {req['params']['name']}")
            print(f"      입력: {req['params']['arguments']}")
            print(f"      출력: {resp[:100]}")

# 실행
data_analysis_scenario()

### 6.3 시나리오 3: 개발자 어시스턴트

GitHub 저장소 정보 + Wikipedia 설명

In [ ]:
def developer_assistant_scenario():
    """개발자 어시스턴트 시나리오"""
    query = "Tell me about the huggingface/transformers GitHub repository"

    print("👨‍💻 개발자 어시스턴트 시나리오")
    print("="*60)
    print(f"📝 질문: {query}")
    print("="*60)

    result = mcp_agent_app.invoke({
        "user_query": query,
        "conversation_history": [],
        "tool_results": [],
        "final_answer": "",
        "iteration": 0,
        "max_iterations": 3
    })

    print("\n" + "="*60)
    print("💬 답변:")
    print(result["final_answer"])

    if result["tool_results"]:
        print(f"\n🔧 사용된 도구:")
        for tool_result in result["tool_results"]:
            req = tool_result['request']
            print(f"   - {req['params']['name']}")

# 실행
developer_assistant_scenario()

## Part 7: MCP의 장단점 분석 ⚖️

### 7.1 MCP의 장점 ✅

#### 1. **표준화**
- 모든 도구가 동일한 JSON 형식 사용
- 새로운 도구 추가가 쉬움 (어댑터만 작성)
- LLM이 하나의 형식만 학습하면 됨

#### 2. **다대다 매칭 (Many-to-Many Architecture)** ⭐

**핵심**: 여러 클라이언트가 여러 MCP 서버를 공유!

```
N:1 패턴 (여러 클라이언트 → 하나의 서버)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Claude Desktop  ─┐
LangChain App   ─┼─→ Wikipedia MCP Server
Custom Agent    ─┘

장점:
✓ 서버 한 번만 구현/유지보수
✓ 모든 앱이 동일한 동작 보장
✓ API 키 중앙 관리
```

```
1:N 패턴 (하나의 클라이언트 → 여러 서버)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
                 ┌─→ Wikipedia MCP
My Agent ────────┼─→ GitHub MCP
                 ├─→ Slack MCP
                 └─→ Database MCP

장점:
✓ 도구 조합 자유로움
✓ 각 서버 독립 업데이트
✓ 팀별 서버 개발 가능
```

```
N:M 패턴 (실제 프로덕션)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
App A ─┐     ┌─→ Server 1
App B ─┼─────┼─→ Server 2
App C ─┘     └─→ Server 3

장점:
✓ 최대 재사용성
✓ 서버 업데이트 시 모든 앱 자동 혜택
✓ 도구 마켓플레이스 가능 (Smithery!)
```

**실제 예시**:

```python
# 기존 방식 (Without MCP): 각 앱마다 구현
# App 1
import wikipedia
result = wikipedia.summary("Python")

# App 2
import wikipedia
result = wikipedia.summary("Python")

# App 3
import wikipedia
result = wikipedia.summary("Python")

# → Wikipedia API 업데이트 시 3개 앱 모두 수정 필요!


# MCP 방식: 서버 하나만 구현
# Wikipedia MCP Server (한 번만!)
@mcp.tool
def search_wikipedia(topic: str) -> str:
    return wikipedia.summary(topic)

# App 1, 2, 3 모두 MCP 서버에 연결만
mcp_client.call_tool("search_wikipedia", {"topic": "Python"})

# → 서버 업데이트 시 모든 앱 자동 혜택!
```

**팀 협업 예시**:

```
도구 개발팀    → Wikipedia MCP Server 개발/유지보수
               → GitHub MCP Server 개발/유지보수
               → DB MCP Server 개발/유지보수

Agent 개발팀  → MCP 서버 사용만 하면 됨
               → 도구 내부 구현 몰라도 됨
               → API 키 관리 불필요
```

#### 3. **재사용성**
- MCP 서버를 여러 애플리케이션에서 공유 가능
- Anthropic Claude Desktop, LangChain 등 다양한 클라이언트 지원
- 한 번 구현하면 모든 프로젝트에서 활용

#### 4. **보안**
- API 키를 MCP 서버에서 중앙 관리
- 클라이언트는 키를 직접 알 필요 없음
- 민감한 정보 격리

#### 5. **멀티 LLM 지원**
- OpenAI, Anthropic, HuggingFace 모델 모두 사용 가능
- LLM 교체 시 코드 변경 최소화

#### 6. **디버깅**
- JSON 형식이므로 요청/응답 추적 쉬움
- 도구 호출 로그 명확함

### 7.1.5 실습: 다대다 매칭 테스트 🧪

이제 **N:1, 1:N, N:M 패턴**을 실제로 테스트해봅시다!

#### 테스트 1: N:1 (다대일) - 여러 Agent가 하나의 MCP Handler 공유

3개의 다른 Agent가 동일한 MCP Handler를 사용합니다.

In [ ]:
print("🧪 테스트 1: N:1 (다대일) 패턴")
print("="*60)
print("여러 Agent → 하나의 MCP Handler\n")

# 공유 MCP Handler (이미 생성됨)
shared_handler = mcp_handler

# Agent 1: Weather Agent
def weather_agent_n1(query: str):
    """날씨만 조회하는 Agent"""
    print("🌤️  [Weather Agent] 요청 처리 중...")

    mcp_request = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": "get_weather",
            "arguments": {"city": query}
        }
    }

    response = shared_handler.handle_request(mcp_request)
    return response['result']['content'][0]['text']

# Agent 2: Exchange Rate Agent
def exchange_agent_n1(from_currency: str, to_currency: str):
    """환율만 조회하는 Agent"""
    print("💱 [Exchange Agent] 요청 처리 중...")

    mcp_request = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": {
            "name": "get_exchange_rate",
            "arguments": {"from_currency": from_currency, "to_currency": to_currency}
        }
    }

    response = shared_handler.handle_request(mcp_request)
    return response['result']['content'][0]['text']

# Agent 3: Wikipedia Agent
def wiki_agent_n1(topic: str):
    """Wikipedia만 검색하는 Agent"""
    print("📚 [Wikipedia Agent] 요청 처리 중...")

    mcp_request = {
        "jsonrpc": "2.0",
        "id": 3,
        "method": "tools/call",
        "params": {
            "name": "search_wikipedia",
            "arguments": {"topic": topic, "sentences": 2}
        }
    }

    response = shared_handler.handle_request(mcp_request)
    return response['result']['content'][0]['text']

# 테스트
print("\n📍 3개 Agent가 동일한 Handler 사용:\n")

result1 = weather_agent_n1("seoul")
print(f"   결과: {result1}\n")

result2 = exchange_agent_n1("USD", "KRW")
print(f"   결과: {result2}\n")

result3 = wiki_agent_n1("Python")
print(f"   결과: {result3[:100]}...\n")

print("="*60)
print("✅ N:1 패턴: 3개 Agent가 하나의 Handler를 성공적으로 공유!")
print("   장점: Handler 업데이트 시 모든 Agent 자동 혜택")

#### 테스트 2: 1:N (일대다) - 하나의 Agent가 여러 도구 세트 사용

하나의 Agent가 여러 카테고리의 도구를 사용합니다.

In [ ]:
print("\n🧪 테스트 2: 1:N (일대다) 패턴")
print("="*60)
print("하나의 Agent → 여러 도구 세트\n")

# 멀티 도구 Agent
def multi_tool_agent(task_type: str, **kwargs):
    """여러 도구를 사용할 수 있는 Agent"""
    print(f"🤖 [Multi-Tool Agent] {task_type} 작업 시작...")

    results = []

    if task_type == "travel_info":
        # 여행 정보: Wikipedia + Weather + Exchange Rate
        print("   1️⃣ Wikipedia 검색")
        wiki_req = create_tools_call_request("search_wikipedia", {"topic": kwargs['destination'], "sentences": 2}, 1)
        wiki_res = mcp_handler.handle_request(wiki_req)
        results.append(("Wikipedia", wiki_res['result']['content'][0]['text'][:80]))

        print("   2️⃣ 날씨 조회")
        weather_req = create_tools_call_request("get_weather", {"city": kwargs['destination'].lower()}, 2)
        weather_res = mcp_handler.handle_request(weather_req)
        results.append(("Weather", weather_res['result']['content'][0]['text']))

        print("   3️⃣ 환율 조회")
        exchange_req = create_tools_call_request("get_exchange_rate", {"from_currency": "USD", "to_currency": "KRW"}, 3)
        exchange_res = mcp_handler.handle_request(exchange_req)
        results.append(("Exchange", exchange_res['result']['content'][0]['text']))

    elif task_type == "developer_info":
        # 개발자 정보: GitHub + Wikipedia + Time
        print("   1️⃣ GitHub 저장소 조회")
        github_req = create_tools_call_request("get_github_repo", {"owner": kwargs['owner'], "repo": kwargs['repo']}, 4)
        github_res = mcp_handler.handle_request(github_req)
        results.append(("GitHub", github_res['result']['content'][0]['text'][:80]))

        print("   2️⃣ 현재 시간 조회")
        time_req = create_tools_call_request("get_current_time", {}, 5)
        time_res = mcp_handler.handle_request(time_req)
        results.append(("Time", time_res['result']['content'][0]['text']))

    return results

# 테스트 1: 여행 정보 (3개 도구)
print("\n📍 Task 1: 여행 정보 수집\n")
travel_results = multi_tool_agent("travel_info", destination="Seoul")
for tool, result in travel_results:
    print(f"   [{tool}] {result}")

# 테스트 2: 개발자 정보 (2개 도구)
print("\n\n📍 Task 2: 개발자 정보 수집\n")
dev_results = multi_tool_agent("developer_info", owner="huggingface", repo="transformers")
for tool, result in dev_results:
    print(f"   [{tool}] {result}")

print("\n" + "="*60)
print("✅ 1:N 패턴: 하나의 Agent가 5개 도구를 성공적으로 조합!")
print("   장점: Agent가 작업에 따라 도구를 자유롭게 선택")

#### 테스트 3: N:M (다대다) - 실전 시뮬레이션

여러 Agent가 여러 도구를 공유하는 실제 시나리오입니다.

In [ ]:
print("\n🧪 테스트 3: N:M (다대다) 패턴")
print("="*60)
print("3개 Agent × 12개 도구 (완전 연결)\n")

# 시뮬레이션: 3개 Agent가 각자 필요한 도구 사용
agents_tasks = [
    ("Travel Agent", ["search_wikipedia", "get_weather", "get_exchange_rate"]),
    ("Data Analyst", ["calculate", "get_current_time", "get_dummy_post"]),
    ("Dev Assistant", ["get_github_repo", "search_wikipedia", "get_country_info"])
]

# 각 Agent가 도구 사용
print("📊 Agent별 도구 사용 현황:\n")

usage_matrix = []

for agent_name, tools_used in agents_tasks:
    print(f"🤖 {agent_name}:")
    for tool in tools_used:
        # 실제 도구 호출
        if tool == "calculate":
            req = create_tools_call_request(tool, {"expression": "10+20"}, 1)
        elif tool == "get_weather":
            req = create_tools_call_request(tool, {"city": "london"}, 1)
        elif tool == "get_exchange_rate":
            req = create_tools_call_request(tool, {"from_currency": "EUR", "to_currency": "USD"}, 1)
        elif tool == "search_wikipedia":
            req = create_tools_call_request(tool, {"topic": "AI", "sentences": 1}, 1)
        elif tool == "get_github_repo":
            req = create_tools_call_request(tool, {"owner": "openai", "repo": "gpt-3"}, 1)
        elif tool == "get_country_info":
            req = create_tools_call_request(tool, {"country_name": "France"}, 1)
        elif tool == "get_current_time":
            req = create_tools_call_request(tool, {}, 1)
        elif tool == "get_dummy_post":
            req = create_tools_call_request(tool, {"post_id": 1}, 1)
        else:
            continue

        res = mcp_handler.handle_request(req)
        result = res['result']['content'][0]['text'][:50]
        print(f"   ✓ {tool}: {result}...")
        usage_matrix.append((agent_name, tool))

    print()

# 매트릭스 시각화
print("="*60)
print("📈 N:M 매트릭스 (Agent × Tool):\n")
print("Agent              | Tools Used")
print("-" * 60)
for agent_name, tools_used in agents_tasks:
    tools_str = ", ".join(tools_used)
    print(f"{agent_name:18} | {tools_str}")

print("\n" + "="*60)
print("✅ N:M 패턴: 3개 Agent가 9개 도구를 공유!")
print("   • 총 도구 호출: 9회")
print("   • Handler 재사용: 9회")
print("   • 장점: 최대 유연성 + 재사용성")

#### 비교: MCP vs 직접 구현

MCP를 사용했을 때와 사용하지 않았을 때의 코드량 비교입니다.

In [ ]:
print("\n📊 코드량 비교: MCP vs 직접 구현")
print("="*60)

comparison = """
시나리오: 3개 Agent가 각각 3개 도구 사용

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
❌ Without MCP (직접 구현)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Agent 1:
  import wikipedia
  def search(topic): ...           # 10줄
  import requests
  def get_weather(city): ...       # 15줄
  def get_exchange(): ...          # 10줄

Agent 2:
  import wikipedia
  def search(topic): ...           # 10줄 (중복!)
  def calculate(expr): ...         # 5줄
  def get_time(): ...              # 5줄

Agent 3:
  import requests
  def get_github(owner, repo): ... # 20줄
  import wikipedia
  def search(topic): ...           # 10줄 (중복!)
  def get_country(name): ...       # 15줄

총 코드량: ~100줄 (중복 포함)
유지보수: 각 Agent 별도 수정 필요

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ With MCP (이번 노트북 방식)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 1. MCP Handler (한 번만!)
mcp_handler = MCPHandler(...)      # 50줄

# 2. Agent들은 단순히 호출
Agent 1:
  mcp_handler.call("search_wikipedia", {...})   # 3줄
  mcp_handler.call("get_weather", {...})        # 3줄
  mcp_handler.call("get_exchange_rate", {...})  # 3줄

Agent 2:
  mcp_handler.call("search_wikipedia", {...})   # 3줄
  mcp_handler.call("calculate", {...})          # 3줄
  mcp_handler.call("get_current_time", {...})   # 3줄

Agent 3:
  mcp_handler.call("get_github_repo", {...})    # 3줄
  mcp_handler.call("search_wikipedia", {...})   # 3줄
  mcp_handler.call("get_country_info", {...})   # 3줄

총 코드량: ~77줄
유지보수: Handler만 수정하면 모든 Agent 자동 업데이트!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💡 결론
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

코드 감소: 23% 절약
중복 제거: 100%
유지보수: 1곳만 수정 vs 3곳 수정
확장성: 도구 추가 시 Handler만 수정

→ Agent가 많아질수록 MCP의 가치는 기하급수적 증가!
"""

print(comparison)

### 7.2 MCP의 단점 ❌

#### 1. **복잡도 증가**
- 간단한 API 호출도 MCP 형식으로 래핑 필요
- 어댑터 코드 작성 오버헤드

```python
# 직접 호출 (3줄)
import requests
response = requests.get("https://api.example.com/weather")
print(response.json())

# MCP 방식 (10+ 줄)
# 1. Tool Definition 작성
# 2. Adapter 함수 작성
# 3. MCP Handler에 등록
# 4. MCP 요청 생성
# 5. Handler 호출
# 6. 응답 파싱
```

#### 2. **JSON 파싱 오류**
- LLM이 잘못된 JSON 생성 가능
- 특히 작은 모델(2B-7B)에서 빈번

```python
# LLM이 생성한 잘못된 JSON
{
  "jsonrpc": "2.0",
  "method": "tools/call"  # id 누락!
  # params 누락!
}
```

#### 3. **성능 오버헤드**
- JSON 직렬화/역직렬화 비용
- MCP 서버와 클라이언트 간 통신 오버헤드 (별도 프로세스 시)

#### 4. **Tool Calling 의존**
- 실제 MCP는 LLM의 tool calling 기능 필요
- HuggingFace 작은 모델들은 tool calling 약함
- → Manual Graph 패턴으로 우회 (이번 노트북의 접근)

#### 5. **도구 수 제한**
- 너무 많은 도구 → LLM 프롬프트 길어짐
- Context length 제한으로 10-20개 도구가 한계

### 7.3 언제 MCP를 사용해야 하는가?

#### ✅ MCP 사용 권장:
1. **도구가 많을 때** (10개 이상)
2. **여러 LLM을 사용할 때** (OpenAI ↔ Anthropic 전환)
3. **도구를 재사용할 때** (여러 프로젝트에서 공유)
4. **팀 협업** (도구 개발자 ≠ Agent 개발자)
5. **보안이 중요** (API 키 중앙 관리)

#### ❌ MCP 불필요:
1. **도구가 1-3개뿐**
2. **프로토타입/POC** (빠른 개발 필요)
3. **단일 LLM만 사용** (LangChain tool 사용이 더 간단)
4. **도구가 자주 변경됨** (어댑터 수정 오버헤드)

### 7.4 실제 MCP 사용 (참고)

이번 노트북은 **교육 목적**으로 MCP 형식을 직접 구현했습니다. 실제 프로덕션에서는:

#### FastMCP (Python)
```bash
pip install fastmcp

# server.py
from fastmcp import FastMCP

mcp = FastMCP("My Server")

@mcp.tool
def search(query: str) -> str:
    return f"Results for {query}"

mcp.run()
```

#### Smithery (Remote MCP)
- 클라우드 호스팅 MCP 서버
- API 키로 접근
- https://smithery.ai

#### Claude Desktop
- Anthropic의 공식 MCP 클라이언트
- 로컬 MCP 서버 연결 가능
- ~/.claude/config.json 설정

### 7.5 결론

**MCP는 도구를 표준화하는 훌륭한 프로토콜입니다.**

하지만:
- 소규모 프로젝트에는 오버킬
- Tool calling 지원 필요
- Manual Graph 패턴으로 우회 가능 (이번 노트북)

**추천**:
1. 먼저 직접 API 호출로 프로토타입 개발
2. 도구가 10개 이상 되면 MCP 도입 고려
3. HuggingFace 모델 사용 시 Manual Graph 패턴 사용

## 📚 요약 및 다음 단계

### ✅ 학습한 내용

1. **MCP JSON-RPC 2.0 정확한 형식**
   - tools/list, tools/call
   - 요청/응답 구조
   - 에러 처리

2. **어댑터 패턴**
   - 실제 API는 MCP를 모름
   - 어댑터로 MCP 형식으로 래핑
   - 12개 무료 API 통합

3. **MCP Handler**
   - 통합 요청 처리기
   - Tool Registry
   - 자동 라우팅

4. **Manual Graph 패턴**
   - Tool calling 없이 MCP 사용
   - EXAONE이 MCP JSON 생성
   - HuggingFace 모델 호환

5. **실전 시나리오**
   - 여행 플래너
   - 데이터 분석
   - 개발자 어시스턴트

6. **장단점 분석**
   - 표준화 vs 복잡도
   - 재사용성 vs 오버헤드
   - 언제 사용할지 판단 기준

### 🎯 핵심 교훈

**MCP의 본질**: 다양한 도구를 **JSON 형식으로 표준화**하는 것!

- Tool calling 필수 아님 (Manual Graph 가능)
- 작은 프로젝트에는 불필요할 수 있음
- 도구 수가 많을수록 MCP의 가치 증가

### 🚀 다음 노트북: 06-advanced-rag-agent.ipynb

Day 2의 RAG + Day 3의 Agent = **Agentic RAG**

- Self-RAG: Agent가 검색 품질 평가
- CRAG (Corrective RAG): 실패 시 재시도
- Query Routing: 쿼리에 따라 전략 선택
- Multi-step RAG: 복잡한 질문 분해